# LL Resummation, starting from a Les Houches Event File

If you use the package ngl\_resum, please cite [doi:10.1007/JHEP09(2020)029](https://inspirehep.net/literature/1798660).

In this example, we will use the ngl\_resum package to resum the non-global logarithms of a rapidity gap, starting from a Les Houches Event File (LHEF) which we read in using the package pylhe. We will use the file sample\_events\_100.lhe in this example, to be found under [https://github.com/MarcelBalsiger/ngl_resum/tree/master/docs/lhef](https://github.com/MarcelBalsiger/ngl_resum/tree/master/docs/lhef). It contains a set of 100 events, created by MadGraph via <code>generate p p > t t~ > vl l+ vl~ l- b b~</code>, stripped of all non-necessary information such as the run card and parameter card. A similar code was used to obtain the results of Section 5 [doi:10.1007/JHEP09(2020)029](https://inspirehep.net/literature/1798660).<br>It makes sense to first have a look at the other example, where we resum the non-global logarithms of a similar rapidity gap but start from one single dipole, as we will not go into every detail here. We suggest this notebook to be used in Binder:
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/MarcelBalsiger/ngl_resum/master?filepath=%2Fdocs%2Fresummator_lhe.ipynb) 

To have this example working as a jupyter notebook, one needs to have the packages numpy, physt, pylhe and - obviously - ngl\_resum installed. The easiest way to do this is to use <code>pip install numpy physt pylhe ngl_resum </code>. Details may be found here: [https://packaging.python.org/tutorials/installing-packages/#use-pip-for-installing](https://packaging.python.org/tutorials/installing-packages/#use-pip-for-installing).

Lastly, one can also download the file resummator\_LHE.py from [https://github.com/MarcelBalsiger/ngl_resum/tree/master/examples](https://github.com/MarcelBalsiger/ngl_resum/tree/master/examples) and play around with it, using this documentation on the side to explain the important steps.

### Imports and constants

We start by importing the packages we need. We will also define some constants, the meaning of which you know from the other example.

In [1]:
import time
import numpy as np
import argparse
import pylhe
import ngl_resum as ngl

nbins = 10
tmax  = 0.1
nsh = 20
showerCutoff = 6.

Note, that <code>nsh</code> is the number of times we want to shower each event. 

### Reading in the LHEF

We will now read in the event file. To do this, we depend entirely on the package pylhe:

In [2]:
pathToFile="lhef/sample_events_100.lhe"
evtFile = pylhe.readLHE(pathToFile)

We have now all our events stored in <code>evtFile</code> and can iterate over each of them. However, before we do that we want to write a method which can check whether the <code>Event</code> needs to be showered (i.e. is valid) and also define an <code>OutsideRegion</code>.

### Check validity of event

We assume an event to be valid, if it fulfills the requirements of Table 1 [https://arxiv.org/abs/1203.5015](https://arxiv.org/abs/1203.5015). This might seem a bit complicated but the following method should be self-explanatory and provide an easy example of how to use the attributes of the <code>Event</code> class when instantiated from an <code>pylhe.LHEEvent</code>.

Let us point out that these attributes of the <code>Event</code> class, accessible by <code>Event.statusType</code> (where <code>status</code> is either <code>incoming</code>, <code>intermediate</code> or <code>outgoing</code>) containes both the particles and antiparticles of <code>Type</code>. For example <code>Event.outgoingBottom</code> is an array of all outgoing bottom quarks and bottom anti-quarks of this instance of <code>Event</code>. This is owed to that fact that in terms of showering the information on the charge can be ignored.

The following method returns <code>True</code> if all conditions are met and returns <code>False</code> if any one of them is violated. 

In [3]:
def validEvent(ev): # ev is the ngl.Event we want to test
    
    # check whether we have the necessary particles
    if ev.intermediateTop == None : return False
    if ev.outgoingBottom == None : return False
    if (ev.outgoingElectron == None) and (ev.outgoingMuon == None): return False
    if len(ev.intermediateTop) != 2 :  return False
    if len(ev.outgoingBottom) != 2 : return False

    momentaLeptonsOut=[]
    momentaNeutrinoOut=[]
    
    electronmuonevent=True
    if not ev.outgoingElectron==None:
        for i in ev.outgoingElectron:
            momentaLeptonsOut.append(i)
            # checks on electron(s)
            if i.eT< 25: return False
            if abs(i.rap)>2.47: return False
        for i in ev.outgoingENeutrino:
            momentaNeutrinoOut.append(i)
    else:
        electronmuonevent=False
            

    if not ev.outgoingMuon==None:
        for i in ev.outgoingMuon:
            momentaLeptonsOut.append(i)
            # checks on muon(s)
            if i.pT< 20: return False
            if abs(i.rap)>2.5: return False
        for i in ev.outgoingMNeutrino:
            momentaNeutrinoOut.append(i)
    else:
        electronmuonevent=False
    
    # check number of leptons ans neutrinos
    if len(momentaLeptonsOut) != 2 : return False
    if len(momentaNeutrinoOut) != 2 : return False
    
    dileptonmass=np.sqrt((momentaLeptonsOut[0]+momentaLeptonsOut[1])*\
                            (momentaLeptonsOut[0]+momentaLeptonsOut[1]))
    missingMomentum=(momentaNeutrinoOut[0]+momentaNeutrinoOut[1])
        
    if not electronmuonevent:
        # checks on "missing momenta" (neutrinos) and dilepton mass
        if missingMomentum.eT<40 : return False
        if (dileptonmass<15 or abs(dileptonmass-91)<10) : return False
    else:
        # check on visible transverse momentum
        if (momentaLeptonsOut[0].pT+momentaLeptonsOut[1].pT+\
            ev.outgoingBottom[0].pT+ev.outgoingBottom[1].pT)<130:
                return False

    # checks on bottom quarks
    for i in ev.outgoingBottom:
        if i.pT<25: return False
        if abs(i.rap)>2.4: return False
        for j in momentaLeptonsOut:
                    if i.R2(j)<0.4**2: return False

    return True # only gets reached, if no check failed.

### Defining the outside region

Now we want to define the outside region to be a rapidity gap, just as in the other example. However, this time we have to account for the jets formed from the bottom quarks and cut them away from the outside region. This means that the region where we veto radiation looks as follows:

<img src="./figures/outside_LHE.JPG" width=380px>

Because this outside region depends on the directions of the bottom quarks (which are treated as the center of the jets), we can not simply instantiate an <code>OutsideRegion</code> once and for all, but have to postpone the instantiation to each event. The <code>_outside(self,vec)</code> method however is the same each time. With <code>self.event</code> you can access the <code>Event</code> you initiate the <code>OutsideRegion</code> with and all the attributes that come with it:

In [4]:
def _outside(self,v):
    jetaxis1=self.event.outgoingBottom[0]/self.event.outgoingBottom[0].e
    jetaxis2=self.event.outgoingBottom[1]/self.event.outgoingBottom[1].e
    jetRadius=0.4
    rapRangeMax=0.8
    rapRangeMin=0.0
    return (v.R2(jetaxis1)>jetRadius**2) and \
           (v.R2(jetaxis2)>jetRadius**2) and \
           (abs(v.rap)<rapRangeMax) and (abs(v.rap)>=rapRangeMin)

### Showering the events, one by one

As we can only shower the LHE file event-by-event, we need a way to keep track of the past showerings. In the end, we want to extract the histogram of the LL resummation and both the one-loop and two-loop coefficients of the expansion, exactly as in the other example. Note that we explicitly do not calculate the error of these quantities in this notebook to streamline the essential discussion, for the calculation with the errors we refer to resummator\_LHE.py from [https://github.com/MarcelBalsiger/ngl_resum/tree/master/examples](https://github.com/MarcelBalsiger/ngl_resum/tree/master/examples). Therefore, we need to set up the following variables: 

In [5]:
fullResultLL=ngl.Hist(nbins,tmax,errorHistCalc=False)
fullNGL1Loop=0.
fullNGL2Loop=0.

We also want to keep track on the number of events, and especially the number of valid events:

In [6]:
numberEvents=0
numberValidEvents=0

Now we all set and ready to shower our events one by one by iterating over <code>evtFile</code>. 

Each time we instantiate an <code>Event</code> with the <code>pylhe.LHEEvent</code>, where the color dipoles get formed between the incoming and intermediate particles and also adding the dipoles formed by the decay of the top quarks to get ready to shower. Note that this setup is specific for the top quark production and decay, as discussed in Section 5 of  [doi:10.1007/JHEP09(2020)029](https://inspirehep.net/literature/1798660)(see Figure 7). You most likely simply need to leave the default values <code>productionDipoles='outgoing',decayDipoles=False</code>. More details on the options <code>productionDipoles</code> and <code>decayDipoles</code> can be found in the documentation of ngl\_resum itself.

After we set up the <code>Event</code>, we check whether it is valid or not using above method. 

If it passes all the tests and is indeed valid, we set up the event-specific <code>OutsideRegion</code>. To do so, we instantiate it by passing the <code>Event</code>, and then replace the stub-method <code>outside(self,v)</code> by above method <code>_outside(self,v)</code> using the same command as in the other example.

With the <code>Event</code> and the <code>OutsideRegion</code> ready, we can start the showering of the event. To do so, we simply instantiate a <code>Shower</code> using the relevant parameters. One should take special care about the parameters <code>nbins</code> and <code>tmax</code> to be the same as in the <code>Hist</code> of <code>fullResultLL</code> as defined above. 

What is then left to do is simply to shower the <code>Shower</code> and add the result to the <code>fullResultLL</code>, as well as adding the one-loop and two-loop coefficients of the expansion. For testing purposes we can track the time (with the default values this following step might take roughly one minute, depending on hardware):

In [7]:
timeStart = time.time()
for event in evtFile:
    numberEvents+=1
        
    ev=ngl.Event(eventFromFile=event,productionDipoles='intermediate',
                    decayDipoles=True)
    
    if validEvent(ev):
        numberValidEvents+=1
        
        outsideRegion=ngl.OutsideRegion(ev)
        outsideRegion.outside = _outside.__get__(outsideRegion,ngl.OutsideRegion)
        shower=ngl.Shower(ev,outsideRegion,nsh,nbins,tmax,showerCutoff)
        shower.shower()
        
        fullResultLL+=shower.resLL
        fullNGL1Loop+=shower.ngl1Loop
        fullNGL2Loop+=shower.ngl2Loop

print("runtime=", time.time()-timeStart,"sec")
print("Of", numberEvents,"events,", numberValidEvents,"were valid.")

runtime= 65.53446531295776 sec
Of 100 events, 63 were valid.


Done!

### Results

Now we have all the information on the showered events stored in the three variables <code>full...</code>. Right now, these are just the sum of all the quantities of every valid event - to get the Monte Carlo estimate, we still have to divide them by this number:

In [8]:
fullResultLL/=numberValidEvents
fullNGL1Loop/=numberValidEvents
fullNGL2Loop/=numberValidEvents

Now we can look at the results of the resummation:

In [9]:
fullResultLL

    t   | entries  
--------|----------
 0.0050 | 0.786765
 0.0150 | 0.518537
 0.0250 | 0.266535
 0.0350 | 0.192396
 0.0450 | 0.108986
 0.0550 | 0.074588
 0.0650 | 0.033867
 0.0750 | 0.033874
 0.0850 | 0.008766
 0.0950 | 0.033986

The one-loop expansion reads

In [10]:
round(shower.ngl1Loop,4)

-30.7448

and the two-loop one

In [11]:
round(shower.ngl2Loop+0.5*shower.ngl1Loop**2,4)

455.3723

Let us stress again that we did not calculate the error of the quantities in this example. A more thorough and less streamlined example code is given by resummator\_LHE.py from [https://github.com/MarcelBalsiger/ngl_resum/tree/master/examples](https://github.com/MarcelBalsiger/ngl_resum/tree/master/examples). Note that several shortcuts we did, such as the division of the <code>Hist</code> by a scalar, do not work when calculating the errors.